In [1]:
from src.data_extraction import *

In [ ]:
code = "CMRY"

client = get_bigquery_client(PROJECT_ID)

df_all, df_quarter = get_all_and_quarterly(code, client)

category = df_all[(df_all['metric']=='market_cap')&
                     (df_all['code']!=code)][['code','clean_value']]\
                            .drop_duplicates().sort_values('clean_value', ascending = False).head(5)['code'].unique()

category = [code] + list(category)

df_all_filtered = df_all[df_all['code'].isin(category)][['code','metric','clean_value']].drop_duplicates()
df_all_filtered = df_all_filtered.pivot(index = 'metric', columns = 'code', values = 'clean_value')

[DEBUG] GOOGLE_APPLICATION_CREDENTIALS = keys/silver-ripple-355716-39ea3374201d.json


c:\Users\hadip\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1962: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [ ]:

# df_all_filtered.to_clipboard()

In [18]:
category

['CMRY', 'ICBP', 'PGUN', 'CPIN', 'INDF', 'MYOR']

In [ ]:
import pandas as pd
import numpy as np
from src.data_extraction import *
import streamlit as st
import altair as alt


code = "CMRY"

client = get_bigquery_client(PROJECT_ID)

df_all, df_quarter = get_all_and_quarterly(code, client)


# assume df is your dataframe
quarter_order = {"Q1": 1, "Q2": 2, "Q3": 3, "Q4": 4}

df = df_quarter[df_quarter['year'].isin([2023, 2024, 2025])].copy()
df["quarter_num"] = df["quarter"].map(quarter_order)
df["period"] = df["year"].astype(str) + " " + df["quarter"]

# sort properly
df = df.sort_values(["year", "quarter_num"])

st.title("CMRY Quarterly Fundamentals")

# Let user choose parameter
param = st.selectbox(
    "Select parameter to plot",
    sorted(df["parameter"].unique())
)

filtered = df[df["parameter"] == param]

st.write(f"Showing **{param}** over time")

line_chart = (
    alt.Chart(filtered)
    .mark_line(point=True)
    .encode(
        x=alt.X("period", sort=list(filtered["period"].unique()), title="Period"),
        y=alt.Y("value_final:Q", title=param),
        tooltip=["period", "year", "quarter", "value_final"]
    )
)

st.altair_chart(line_chart, use_container_width=True)


2025-12-01 20:49:42.395 
  command:

    streamlit run c:\Users\hadip\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-12-01 20:49:42.400 Session state does not function when running a script without `streamlit run`


DeltaGenerator()

In [5]:
df_all

,int64_field_0,code,metric,value,clean_value,latest_data_datetime
0,21006,CMRY,Cash From Financing (TTM),"(1,193 B)",-1.193000e+12,2025-09-30
1,21003,CMRY,Net Debt (Quarter),"(1,215 B)",-1.215000e+12,2025-09-30
2,21007,CMRY,Capital expenditure (TTM),(477 B),-4.770000e+11,2025-09-30
3,21005,CMRY,Cash From Investing (TTM),(879 B),-8.790000e+11,2025-09-30
4,20952,CMRY,Debt to Equity Ratio (Quarter),-,NaN,2025-09-30
...,...,...,...,...,...,...
89,20990,CMRY,Rank (Near 52 Weeks High),89.00%,8.900000e-01,2025-09-30
90,20959,CMRY,Altman Z-Score (Modified),9.22,9.220000e+00,2025-09-30
91,20948,CMRY,Current Book Value Per Share,902.72,9.027200e+02,2025-09-30
92,20979,CMRY,Payout Ratio,92.92%,9.292000e-01,2025-09-30
